In [1]:
import os
import sqlite3
import pandas as pd
import tqdm
import numpy as np
db_name = '[TFIDF]Credibility_research_20180906.db' #DB 파일명
# db 생성
con = sqlite3.connect( db_name )
cur = con.cursor()

In [2]:
sql = '''
select p.Post_id,p.Category,u.Blog_nickname, u.User_id, p.Title, p.Structure,p.Structure_tag, p.Text, p.Map_exist, p.Heart_count, p.Comment_count, u.Blog_name, u.Blog_nickname, u.Blog_info_text,u.Count_neighbors,u.Count_visitors,u.Categories,u.Count_categories,u.Total_post,u.Blog_mobile_profile_img,u.Blog_mobile_cover_img,u.Opening_date,u.Post_interval_average,ps.Align, ps.basic_font,ps.basic_font_size,ps.b_strong_word_len,ps.color_rgb_word_len,ps.b_tag_len,ps.color_tag_len, group_concat(img.Img_id) as img_count,u.Credibility
    FROM Post as p
    Left JOIN user as u
        ON p.User_id = u.User_id
    Left JOIN Post_style as ps
        ON p.Post_id = ps.Post_id
    Left JOIN Img as img
        ON p.Post_id = img.Post_id
    GROUP BY p.Post_id;
'''
cur.execute(sql)
rows = cur.fetchall()

In [3]:
col_name = [i[0] for i in cur.description]

In [4]:
test_df = pd.DataFrame(rows)

In [5]:
test_df.columns = col_name

In [7]:
test_df.columns.tolist()

['Post_id',
 'Category',
 'Blog_nickname',
 'User_Id',
 'Title',
 'Structure',
 'Structure_tag',
 'Text',
 'Map_exist',
 'Heart_count',
 'Comment_count',
 'Blog_name',
 'Blog_nickname',
 'Blog_info_text',
 'Count_neighbors',
 'Count_visitors',
 'Categories',
 'Count_categories',
 'Total_post',
 'Blog_mobile_profile_img',
 'Blog_mobile_cover_img',
 'Opening_date',
 'Post_interval_average',
 'Align',
 'basic_font',
 'basic_font_size',
 'b_strong_word_len',
 'color_rgb_word_len',
 'b_tag_len',
 'color_tag_len',
 'img_count',
 'Credibility']

In [9]:
Category_df = test_df[['Post_id','Category']]
Category_df.to_csv('Category.csv')

In [6]:
test_df.drop_duplicates(subset=['User_Id']).groupby(by='Credibility')['Title'].count()

Credibility
0    1955
1     989
Name: Title, dtype: int64

In [7]:
test_df.groupby(by='Credibility')['User_Id'].count()

Credibility
0    6362
1    9942
Name: User_Id, dtype: int64

In [8]:
pd.DataFrame(test_df[test_df['Credibility']==1].drop_duplicates(subset='User_Id').groupby(by='Category')['User_Id'].count())

,User_Id
Category,
IT·컴퓨터,48
건강·의학,51
공연·전시,51
교육·학문,50
국내여행,48
드라마·방송,27
등산·낚시·레저,23
만화·애니,61
맛집,104


In [9]:
pd.DataFrame(test_df[test_df['Credibility']==1].groupby(by='Category')['Title'].count())

,Title
Category,
IT·컴퓨터,476
건강·의학,565
공연·전시,532
교육·학문,492
국내여행,475
드라마·방송,258
등산·낚시·레저,232
만화·애니,653
맛집,1033


## Question_mark

In [10]:
Question_Mark = test_df['Text'].fillna('').tolist()

In [510]:
count_Question = []
for i in Question_Mark:
    count_Question.append(i.count('?'))

## TFIDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np

In [3]:
content_df = pd.read_csv('Content_based_features_csv.csv',encoding='utf-8',sep='\t')

## Text

In [4]:
loaded_vec_text = pickle.load(open("[Text]TFIDF_features_100.pkl", "rb"))

In [5]:
text = content_df['Embedding_text'].fillna('').tolist()

In [6]:
Post_id_list = content_df['Post_id'].tolist()

In [7]:
vector_text = []
for i in text:
    vector_text.append(loaded_vec_text.transform([i]).toarray())

In [8]:
vector = np.vstack(np.array(vector_text))

In [9]:
colum_name = loaded_vec_text.get_feature_names()

In [10]:
Text_df = pd.DataFrame(vector,columns=colum_name)

In [11]:
Text_df['Post_id'] = Post_id_list

In [12]:
Text_df = pd.merge(content_df,Text_df,how='left')

## Structure_tag

In [13]:
content_df = pd.read_csv('Content_based_features_csv.csv',encoding='utf-8',sep='\t')

In [14]:
loaded_vec_Structure_tag = pickle.load(open("[Structure_tag]TFIDF_features_100.pkl", "rb"))

In [15]:
Post_id_list = content_df['Post_id'].tolist()

In [16]:
content_df['Structure_tag'] = content_df['Structure_tag'].fillna('')

In [17]:
content_df['only_tag'] = content_df['Structure_tag'].apply(lambda x : " img ".join(list(map(lambda x : 'text' if len(x)>3 else '', x.split('<img>')))).strip().replace('  ',' '))

In [18]:
Structure_tag = content_df['only_tag'].tolist()

In [19]:
vector_Structure_tag = []
for i in Structure_tag:
    vector_Structure_tag.append(loaded_vec_Structure_tag.transform([i]).toarray())

In [20]:
vector = np.vstack(np.array(vector_Structure_tag))

In [21]:
colum_name = loaded_vec_Structure_tag.get_feature_names()

In [22]:
Structure_Tag_df = pd.DataFrame(vector,columns=colum_name)

In [23]:
Structure_Tag_df['Post_id'] = Post_id_list

In [24]:
Structure_Tag_df = pd.merge(Text_df,Structure_Tag_df,how='left')

## Tag

In [582]:
content_df = pd.read_csv('Content_based_features_csv.csv',encoding='utf-8',sep='\t')

In [26]:
loaded_vec_tag = pickle.load(open("[Tag]features_100.pkl", "rb"))

In [595]:
Tag_count = content_df['Tag'].fillna(0).apply(lambda x: len(x.split(' ')) if x!=0 else x).tolist()

In [588]:
Tag_count = []
for i in Tag:
    try:
        Tag_count.append(len(i.split(' ')))
    except:
        Tag_count.append(0)

In [28]:
Post_id_list = content_df['Post_id'].tolist()

In [29]:
content_df['Tag'] = content_df['Tag'].apply(lambda x : x.replace(',',' '))

In [30]:
Tag = content_df['Tag'].tolist()

In [31]:
vector_tag = []
for i in Tag:
    vector_tag.append(loaded_vec_tag.transform([i]).toarray())

In [32]:
vector = np.vstack(np.array(vector_tag))

In [33]:
colum_name = loaded_vec_tag.get_feature_names()

In [34]:
Tag_df = pd.DataFrame(vector,columns=colum_name)

In [35]:
Tag_df['Post_id'] = Post_id_list

In [36]:
Tag_df = pd.merge(Structure_Tag_df,Tag_df,how='left')

In [37]:
Tag_df.shape

(16304, 244)

In [38]:
Tag_df.columns.tolist()

['Post_id',
 'Title_len',
 'Count_space_mistake',
 'Map_exist',
 'Media_count',
 'Heart_count',
 'Sticker_count',
 'Comment_count',
 'Left',
 'Center',
 'Right',
 'Justify',
 'basic_font_check',
 'basic_font_size_check',
 'b_strong_word_len_ratio',
 'color_rgb_word_len_ratio',
 'First_ratio',
 'Second_ratio',
 'Text_len',
 'Img_count',
 'FR_url',
 'pos_ratio',
 'neg_ratio',
 'subjectivity',
 'polarity',
 'senti_diffs_per_ref',
 'Text',
 'Tag',
 'Structure_tag',
 'Credibility',
 'Embedding_text',
 '가능하다',
 '가다',
 '간단하다',
 '갈다',
 '계시다',
 '궁금하다',
 '그렇다',
 '깔끔하다',
 '나가다',
 '나다',
 '나오다',
 '남다',
 '내다',
 '너무',
 '넘다',
 '넣다',
 '높다',
 '느끼다',
 '늘다',
 '다니다',
 '다르다',
 '다양하다',
 '덥다',
 '돼다',
 '되다',
 '되어다',
 '두다',
 '드리다',
 '들다',
 '들어가다',
 '들이다',
 '따르다',
 '만나다',
 '만들다',
 '많다',
 '많이',
 '말다',
 '맛있다',
 '맞다',
 '먹다',
 '모르다',
 '믿다',
 '바라다',
 '받다',
 '버리다',
 '보내다',
 '보다',
 '보이다',
 '빠르다',
 '빠지다',
 '새롭다',
 '생기다',
 '서다',
 '쉬다',
 '시원하다',
 '싶다',
 '쓰다',
 '아니다',
 '아직',
 '안녕하다',
 '안되다',
 '않다',
 '알다',
 '알아보다',
 '어떻다',


## Total

In [39]:
Content_based_df = Tag_df.drop(columns=['Embedding_text','Text','Tag','Structure_tag'])

In [48]:
colum_name = Content_based_df.columns.tolist()

In [57]:
colum_name.remove('Credibility')

In [59]:
colum_name.append('Credibility')

In [64]:
Total_df = Content_based_df[colum_name]

In [69]:
Total_df.to_csv('[Final]Content_based_csv.csv',encoding='utf-8',sep='\t',index=False)

In [512]:
Total_df['Question_count'] = count_Question

/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [596]:
Total_df['Tag_count'] = Tag_count

/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [598]:
temp_col = Total_df.columns.tolist()[:-2]

In [600]:
temp_col.append('Tag_count')

In [601]:
temp_col.append('Credibility')

In [534]:
temp_list = Total_df.columns.tolist()[:-2]

In [535]:
temp_list.append('Question_count')

In [536]:
temp_list.append('Credibility')

In [602]:
Total_df = Total_df[temp_col]

In [603]:
Total_df.to_csv('[Final]Content_based_csv.csv',encoding='utf-8',sep='\t',index=False)

In [645]:
Total_df.columns.tolist()

['Post_id',
 'Title_len',
 'Count_space_mistake',
 'Map_exist',
 'Media_count',
 'Heart_count',
 'Sticker_count',
 'Comment_count',
 'Left',
 'Center',
 'Right',
 'Justify',
 'basic_font_check',
 'basic_font_size_check',
 'b_strong_word_len_ratio',
 'color_rgb_word_len_ratio',
 'First_ratio',
 'Second_ratio',
 'Text_len',
 'Img_count',
 'FR_url',
 'pos_ratio',
 'neg_ratio',
 'subjectivity',
 'polarity',
 'senti_diffs_per_ref',
 '가능하다',
 '가다',
 '간단하다',
 '갈다',
 '계시다',
 '궁금하다',
 '그렇다',
 '깔끔하다',
 '나가다',
 '나다',
 '나오다',
 '남다',
 '내다',
 '너무',
 '넘다',
 '넣다',
 '높다',
 '느끼다',
 '늘다',
 '다니다',
 '다르다',
 '다양하다',
 '덥다',
 '돼다',
 '되다',
 '되어다',
 '두다',
 '드리다',
 '들다',
 '들어가다',
 '들이다',
 '따르다',
 '만나다',
 '만들다',
 '많다',
 '많이',
 '말다',
 '맛있다',
 '맞다',
 '먹다',
 '모르다',
 '믿다',
 '바라다',
 '받다',
 '버리다',
 '보내다',
 '보다',
 '보이다',
 '빠르다',
 '빠지다',
 '새롭다',
 '생기다',
 '서다',
 '쉬다',
 '시원하다',
 '싶다',
 '쓰다',
 '아니다',
 '아직',
 '안녕하다',
 '안되다',
 '않다',
 '알다',
 '알아보다',
 '어떻다',
 '어렵다',
 '없다',
 '없이',
 '예쁘다',
 '오다',
 '올리다',
 '원하다',
 '유명하다',
 '이렇게',


## User_based_features

In [606]:
sql = '''
select p.Post_id, p.Date,group_concat(DISTINCT cmt.Comment_Time) as Comment_Time, u.blog_info_text, u.Count_neighbors, u.Total_post, u.Count_visitors, u.Categories, u.Count_categories, u.Blog_mobile_profile_img,u.Blog_mobile_cover_img,u.Opening_date,Post_interval_average,p.Text_similarity,group_concat(DISTINCT cmt.Comment_ID) as Comment_ID,u.User_Id,u.Credibility
    FROM POST as p
    Left JOIN Post_style as style
        ON p.Post_id = style.Post_id
    Left JOIN img as i
        ON p.Post_id = i.Post_id
    Left JOIN user as u
        ON p.user_id = u.user_id
    Left JOIN Comment_interaction as cmt
        ON p.Post_id = cmt.Post_id        
    GROUP BY p.Post_id;
'''
cur.execute(sql)
rows = cur.fetchall()

In [607]:
pd.DataFrame(rows)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,100027815366,2006. 8. 25. 0:40,"2007.12.11. 20:58,2006.8.25. 15:01,2007.12.19....",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.232631,"coco2243,hahnn1,kkk1194,simmiani,yhh0924",hahnn1,1
1,100050521880,2008. 5. 8. 1:21,"2008.5.22. 20:48,2008.5.24. 01:15,2008.5.23. 2...",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.112250,"hahnn1,purusego,studiohan,with_dust,yhh0924",hahnn1,1
2,100052476065,2008. 7. 6. 1:01,"2008.7.12. 02:10,2008.7.17. 01:08,2008.7.12. 1...",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.071837,"9810213,bichoobichoo,hahnn1,jhcworld,ppoppopia...",hahnn1,1
3,100052627538,2008. 7. 11. 10:28,"2008.7.25. 22:31,2008.7.24. 23:57,2008.7.24. 1...",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.246814,"0501mansae,9810213,hahnn1,marrian,purusego,yhh...",hahnn1,1
4,100052919717,2008. 7. 20. 23:29,None,자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.227445,None,hahnn1,1
5,100053144381,2008. 7. 27. 14:17,"2010.3.12. 16:32,2008.8.18. 12:22,2008.8.18. 1...",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.221925,"cinetist,hahnn1,idea365,jhcworld,purusego,with...",hahnn1,1
6,100053820465,2008. 8. 17. 18:24,"2008.9.12. 11:15,2008.9.12. 09:56",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.043462,"hahnn1,yhh0924",hahnn1,1
7,100054338281,2008. 8. 31. 0:52,"2008.9.16. 13:59,2008.9.15. 17:23",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.065938,"hahnn1,purusego",hahnn1,1
8,100064414352,2009. 3. 28. 14:13,"2009.3.29. 12:32,2009.3.30. 14:06,2009.3.29. 1...",자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,0.086118,"hahnn1,obvan,yhh0924",hahnn1,1
9,100067528683,2009. 5. 26. 0:42,None,자기소개를 입력하세요.,976,1769,927604,"[(122, ""'Olive'""), (17, ""'올리브호주Aussie'""), (5, ...",64,Naver\hahnn1\Cover_hahnn1.jpg,Naver\hahnn1\Profile_hahnn1.jpg,2005.04.09.,10.666667,NaN,None,hahnn1,1


In [608]:
column_name = [i[0] for i in cur.description]
User_based = pd.DataFrame(rows)
User_based.columns = column_name

In [609]:
User_based.shape

(16304, 17)

In [610]:
User_based['Blog_info_text_len'] = User_based['Blog_info_text'].fillna('').apply(lambda x : len(x))

In [611]:
Comment_id = User_based['Comment_ID'].fillna('').apply(lambda x : x.split(',')).tolist()

In [612]:
User_id = User_based['User_Id'].tolist()

In [613]:
check_list = []
for i , Comment_id in enumerate(Comment_id):
    if User_id[i] in Comment_id:
        check_list.append(1)
    else:
        check_list.append(0)

In [614]:
User_based['Comment_interaction'] = check_list

## User_id charcher

In [615]:
import re
p = re.compile("[^0-9]")
User_id_len = len(User_based['User_Id'].iloc[0])
Char_len = len("".join(p.findall(User_based['User_Id'].iloc[0])))
num_len = User_id_len - Char_len

In [616]:
import re
p = re.compile("[^0-9]")
User_based['User_Id_len'] = User_based['User_Id'].apply(lambda x : len(x))
User_based['User_Id_contain_num'] = User_based['User_Id'].apply(lambda x : len("".join(p.findall(x))))
User_based['User_Id_contain_char'] = User_based['User_Id_len'] - User_based['User_Id_contain_num']

## Time Stamp

In [617]:
from datetime import datetime
import time
def timestamp_encoding(Time):
    Time = Time.replace(' ','')
    timestamp = time.mktime(datetime.strptime(Time,'%Y.%m.%d.%H:%M').timetuple())
    return timestamp

def timestamp_decoding(Time):
    timestamp = datetime.fromtimestamp(timestamp).strftime('%Y.%m,%d.%H:%M')
    return timestamp

In [618]:
User_date = User_based['Date'].tolist()

In [619]:
User_date[0]

'2006. 8. 25. 0:40'

In [620]:
User_date_list = []
for i in User_date:
    try:
        date = i.replace(' ','')
        timestamp = timestamp_encoding(date)
        User_date_list.append(timestamp)
    except:
        User_date_list.append(None)

In [621]:
User_based['Date'] = User_date_list

In [622]:
Comment_time = User_based['Comment_Time'].tolist()

In [623]:
Average_comment_time = []
First_comment_time = []
for i in Comment_time:
    try:
        Comment_string = i.replace(' ','')
        Comment_time_list = Comment_string.split(',')
        Total_comment_time = []
        
        for j in Comment_time_list:
            Total_comment_time.append(timestamp_encoding(j))

        Average_comment_time.append(sum(Total_comment_time)/len(Total_comment_time))
        
        First_comment_time.append(Total_comment_time[-1])
    except:
        Average_comment_time.append(None)
        First_comment_time.append(None)

In [624]:
comment_diff = []
First_comment_diff = []

for i , User_date in enumerate(User_date_list):
    try:
        comment_diff.append(Average_comment_time[i] - User_date )
    except:
        comment_diff.append(None)
    try:
        First_comment_diff.append(First_comment_time[i] - User_date)
    except:
        First_comment_diff.append(None)

In [625]:
User_based.columns.tolist()

['Post_id',
 'Date',
 'Comment_Time',
 'Blog_info_text',
 'Count_neighbors',
 'Total_post',
 'Count_visitors',
 'Categories',
 'Count_categories',
 'Blog_mobile_profile_img',
 'Blog_mobile_cover_img',
 'Opening_date',
 'Post_interval_average',
 'Text_similarity',
 'Comment_ID',
 'User_Id',
 'Credibility',
 'Blog_info_text_len',
 'Comment_interaction',
 'User_Id_len',
 'User_Id_contain_num',
 'User_Id_contain_char']

In [626]:
User_based['Average_comment_diff'] = comment_diff
User_based['First_comment_diff'] = First_comment_diff

In [627]:
def timestamp_encoding_opennig_date(Time):
    Time = Time.replace(' ','')
    timestamp = time.mktime(datetime.strptime(Time,'%Y.%m.%d.').timetuple())
    return timestamp

In [628]:
date = User_based['Opening_date'].tolist()

In [629]:
date_list = []
for i in date:
    try:
        date_list.append(timestamp_encoding_opennig_date(i))
    except:
        date_list.append(None)

In [630]:
User_based['Opening_date'] = date_list

In [631]:
User_df = User_based[['Blog_info_text_len','Count_neighbors','Count_visitors','Post_interval_average','Text_similarity','Total_post','User_Id_len','User_Id_contain_num','User_Id_contain_char','Comment_interaction','Count_categories','Average_comment_diff','First_comment_diff','Opening_date','Blog_mobile_profile_img','Blog_mobile_cover_img','Credibility']]

In [632]:
User_df.columns.tolist()

['Blog_info_text_len',
 'Count_neighbors',
 'Count_visitors',
 'Post_interval_average',
 'Text_similarity',
 'Total_post',
 'User_Id_len',
 'User_Id_contain_num',
 'User_Id_contain_char',
 'Comment_interaction',
 'Count_categories',
 'Average_comment_diff',
 'First_comment_diff',
 'Opening_date',
 'Blog_mobile_profile_img',
 'Blog_mobile_cover_img',
 'Credibility']

## Total_df

In [633]:
col = Total_df.columns.tolist()[:-1]

In [634]:
Total_computational_df = pd.concat([Total_df[col],User_df],axis=1)

In [635]:
Total_computational_df.to_csv('[Final]Total_computational_df.csv',encoding='utf-8',sep='\t',index=False)

In [636]:
Total_computational_df.columns.tolist()

['Post_id',
 'Title_len',
 'Count_space_mistake',
 'Map_exist',
 'Media_count',
 'Heart_count',
 'Sticker_count',
 'Comment_count',
 'Left',
 'Center',
 'Right',
 'Justify',
 'basic_font_check',
 'basic_font_size_check',
 'b_strong_word_len_ratio',
 'color_rgb_word_len_ratio',
 'First_ratio',
 'Second_ratio',
 'Text_len',
 'Img_count',
 'FR_url',
 'pos_ratio',
 'neg_ratio',
 'subjectivity',
 'polarity',
 'senti_diffs_per_ref',
 '가능하다',
 '가다',
 '간단하다',
 '갈다',
 '계시다',
 '궁금하다',
 '그렇다',
 '깔끔하다',
 '나가다',
 '나다',
 '나오다',
 '남다',
 '내다',
 '너무',
 '넘다',
 '넣다',
 '높다',
 '느끼다',
 '늘다',
 '다니다',
 '다르다',
 '다양하다',
 '덥다',
 '돼다',
 '되다',
 '되어다',
 '두다',
 '드리다',
 '들다',
 '들어가다',
 '들이다',
 '따르다',
 '만나다',
 '만들다',
 '많다',
 '많이',
 '말다',
 '맛있다',
 '맞다',
 '먹다',
 '모르다',
 '믿다',
 '바라다',
 '받다',
 '버리다',
 '보내다',
 '보다',
 '보이다',
 '빠르다',
 '빠지다',
 '새롭다',
 '생기다',
 '서다',
 '쉬다',
 '시원하다',
 '싶다',
 '쓰다',
 '아니다',
 '아직',
 '안녕하다',
 '안되다',
 '않다',
 '알다',
 '알아보다',
 '어떻다',
 '어렵다',
 '없다',
 '없이',
 '예쁘다',
 '오다',
 '올리다',
 '원하다',
 '유명하다',
 '이렇게',


## Paper_best_features

In [641]:
Best_feature_df = Total_computational_df[['Post_id','FR_url','Question_count','First_ratio','Second_ratio','Tag_count','Blog_info_text_len','Count_neighbors','Text_len','Post_interval_average','Text_similarity','Total_post','User_Id_len','Comment_interaction','Blog_mobile_profile_img','Opening_date','senti_diffs_per_ref','pos_ratio','neg_ratio','subjectivity','Sticker_count','Credibility']]

In [642]:
Best_feature_df.to_csv('[Final]Best_feautres_df.csv',encoding='utf-8',sep='\t',index=False)

In [3]:
import pandas as pd
Best_feature_df = pd.read_csv('[Final]Best_feautres_df.csv',encoding='utf-8',sep='\t')

In [4]:
Best_feature_df.columns.tolist()

['Post_id',
 'FR_url',
 'Question_count',
 'First_ratio',
 'Second_ratio',
 'Tag_count',
 'Blog_info_text_len',
 'Count_neighbors',
 'Text_len',
 'Post_interval_average',
 'Text_similarity',
 'Total_post',
 'User_Id_len',
 'Comment_interaction',
 'Blog_mobile_profile_img',
 'Opening_date',
 'senti_diffs_per_ref',
 'pos_ratio',
 'neg_ratio',
 'subjectivity',
 'Sticker_count',
 'Credibility']